In [1]:
import pandas as pd
import os
import sqlite3
from DB_insert import find_city, add_article, get_production_method, insert_chelpipe, get_steel, get_production_method
import datetime
from tqdm import tqdm
from natasha import MorphVocab, AddrExtractor
# import DB_insert

In [2]:
type_dict = {'Холоднодеформированные трубы': 'х/д',
             'Горячедеформированные трубы': 'г/д',
             'Электросварные трубы': 'ЭСВ',
             'Не указано': None}

In [3]:
morph_vocab = MorphVocab()
addr_extractor = AddrExtractor(morph_vocab)

In [4]:
path = r'\\chl-vm-aicenter.chtpz.ru\Projects\parsing_price\data'
name = r'\price_td_uts.csv'
# df = pd.read_csv(os.path.join(path, name), comment='#', error_bad_lines=False)
df = pd.read_csv(path + name, comment='#', sep=';', error_bad_lines=False)
df.shape

(13648, 19)

In [5]:
with sqlite3.connect(r'../front/db_test5.sqlite') as connection:
    con = connection
cur = con.cursor()

In [6]:
df[df['ЦенаЗаТонну']>0].head()

,КодНИС,НаименованиеНоменклатуры,ВидДлины,Производитель,ГОСТ,МаркаСтали,Диаметр,Размер2,Стенка,ТипПроизводства,РегионСклада,Склад,ИмеетсяЦенаТовара,ЦенаЗаТонну,ЦенаЗаТонну1,ЦенаЗаТонну2,ЦенаЗаМетр,ЦенаЗаМетр1,ЦенаЗаМетр2
4212,1917267,"ТУ 1319-1128-00186654-2012, СтК48, 426 * 20 ...",НК,ЧТПЗ,ТУ 1319-1128-00186654-2012,К48,426.0,0.0,20.0,Горячедеформированные трубы,66,"Склад г. Екатеринбург ( ООО ""АКОРТЕС"" )",Да,100866.68,100866.68,100866.68,20198.22,20198.22,20198.22
4226,1926916,"ТРУБЫ б/ш г/д общ назн ГОСТ 8732-78, ГОСТ 8731...",НК,ПНТЗ,"8732-78, 8731-74, В",35,102.0,0.0,18.0,Горячедеформированные трубы,50,"Склад г. Москва (АО ""Жилевская металлобаза"") ...",Да,77083.34,76666.67,75833.34,2874.23,2858.70,2827.63
4227,1926920,"ТРУБЫ б/ш г/д общ назн ГОСТ 8732-78, ГОСТ 8731...",НК,ПНТЗ,"8732-78, 8731-74, В",35,114.0,0.0,18.0,Горячедеформированные трубы,66,"Склад г. Екатеринбург ( ООО ""АКОРТЕС"" )",Да,77500.01,77083.34,76250.01,3302.59,3284.84,3249.33
4230,1927042,"ТУ 1319-1128-00186654-2012, СтК48, 325 * 16 ...",НК,ЧТПЗ,ТУ 1319-1128-00186654-2012,К48,325.0,0.0,16.0,Горячедеформированные трубы,66,"Склад г. Екатеринбург ( ООО ""АКОРТЕС"" )",Да,94700.01,94700.01,94700.01,11546.17,11546.17,11546.17
4236,1930879,"Труба б/ш х/д ГОСТ 8734-75, ГОСТ 8733-74 Групп...",НК,ПНТЗ,"8734-75, 8733-74, В",09Г2С,16.0,0.0,2.0,Холоднодеформированные трубы,42,"Склад г. Новокузнецк (ОАО ""Новокузнецкметаллоп...",Да,227332.36,220512.39,210492.93,156.98,152.27,145.35


In [7]:
df.columns

Index(['КодНИС', 'НаименованиеНоменклатуры', 'ВидДлины', 'Производитель',
       'ГОСТ', 'МаркаСтали', 'Диаметр', 'Размер2', 'Стенка', 'ТипПроизводства',
       'РегионСклада', 'Склад', 'ИмеетсяЦенаТовара', 'ЦенаЗаТонну',
       'ЦенаЗаТонну1', 'ЦенаЗаТонну2', 'ЦенаЗаМетр', 'ЦенаЗаМетр1',
       'ЦенаЗаМетр2'],
      dtype='object')

In [8]:
time = datetime.datetime.now().timestamp()
first = True
for index in tqdm(range(df.shape[0])):
    name = df.loc[index]['НаименованиеНоменклатуры']
    standards = df.loc[index]['ГОСТ'].split(',')
#     print(df.loc[index]['МаркаСтали'])
    steel_id = get_steel(df.loc[index]['МаркаСтали'], cur, con)
    diameter = df.loc[index]['Диаметр']
    wall = df.loc[index]['Стенка']
    production_method_id = get_production_method(type_dict[df.loc[index]['ТипПроизводства']], cur, con)
    city_id = find_city(df.loc[index]['Склад'], cur, con, addr_extractor)
    price_min = df.loc[index]['ЦенаЗаТонну'] * 1.2
    price_max = df.loc[index]['ЦенаЗаТонну2'] * 1.2
    article_id = add_article(int(df.loc[index]['КодНИС']), cur, con)
    for standard in standards:
        data = [name, standard.strip(), steel_id, diameter, wall, city_id, price_min, 
                price_max, article_id, time, production_method_id]
        insert_chelpipe(data, cur, con, first)
        if first:
            first = False

100%|██████████| 13648/13648 [1:59:28<00:00,  1.90it/s]  


columns = '(name, standard, steel_id, diameter, wall, city_id, price_min, price_max, article_id, parce_date, method_id)'